# 🌐 Chapter 3D-7: Multiplayer 3D

You built a 3D world. You built a networking system. Now it's time to **combine them**.

This chapter shows you how to sync **multiple first-person cameras** in a shared 3D space.

### What You'll Learn
- How to serialize 3D position and rotation data
- How to render other players as cubes in your world
- How to sync camera movement over the network
- How to build a multiplayer 3D arena

## 1. The Player State (Position + Rotation)

In Chapter M3, we learned to use `pickle` to serialize data. For 3D multiplayer, we need to send:
- **Position**: (X, Y, Z)
- **Rotation**: (Yaw, Pitch)

In [ ]:
import pickle

class PlayerState:
    def __init__(self, player_id):
        self.id = player_id
        self.pos = [0.0, 0.0, 3.0]  # X, Y, Z
        self.yaw = -90.0
        self.pitch = 0.0
    
    def update_from_camera(self, camera):
        """Update state from a Camera object."""
        self.pos = camera.position.tolist()
        self.yaw = camera.yaw
        self.pitch = camera.pitch
    
    def serialize(self):
        """Convert to bytes for network transmission."""
        return pickle.dumps(self)
    
    @staticmethod
    def deserialize(data):
        """Recreate from bytes."""
        return pickle.loads(data)

## 2. The Server (Managing Multiple 3D Players)

The server tracks all connected players and broadcasts their states.

In [ ]:
import socket
import pickle
import threading

players = {}  # {conn: PlayerState}
next_id = 0

def handle_client(conn, addr):
    global next_id
    
    player_id = next_id
    next_id += 1
    
    player = PlayerState(player_id)
    players[conn] = player
    
    print(f"Player {player_id} connected from {addr}")
    
    # Send the player their ID
    conn.send(pickle.dumps(player_id))
    
    try:
        while True:
            # Receive updated player state
            data = conn.recv(4096)
            if not data:
                break
            
            player_state = pickle.loads(data)
            players[conn] = player_state
            
            # Broadcast all player states to this client
            all_states = {p.id: p for p in players.values()}
            conn.sendall(pickle.dumps(all_states))
    
    except Exception as e:
        print(f"Error with player {player_id}: {e}")
    
    finally:
        print(f"Player {player_id} disconnected")
        del players[conn]
        conn.close()

# Start the server
server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind(('0.0.0.0', 5555))
server.listen()

print("[SERVER] Waiting for connections...")

while True:
    conn, addr = server.accept()
    thread = threading.Thread(target=handle_client, args=(conn, addr))
    thread.start()

## 3. The Client (Sending Camera State)

The client sends its camera position every frame and receives all other players.

In [ ]:
import socket
import pickle
import pygame
import moderngl
from pyrr import Matrix44, Vector3
import numpy as np

# Connect to server
client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client.connect(('127.0.0.1', 5555))

# Get our player ID
my_id = pickle.loads(client.recv(2048))
print(f"Connected! You are Player {my_id}")

# ... (Initialize Pygame, ModernGL, Camera, Cube rendering code) ...

other_players = {}  # {player_id: PlayerState}

def send_state(camera):
    """Send our camera state to the server."""
    state = PlayerState(my_id)
    state.update_from_camera(camera)
    client.send(state.serialize())

def receive_states():
    """Get all player states from the server."""
    try:
        data = client.recv(4096)
        if data:
            all_states = pickle.loads(data)
            # Filter out our own state
            return {pid: state for pid, state in all_states.items() if pid != my_id}
    except:
        pass
    return {}

# In the game loop:
# send_state(camera)
# other_players = receive_states()

## 4. Rendering Other Players

We render each other player as a cube at their position.

In [ ]:
def render_player(player_state, prog, vao, projection, view):
    """Render a player as a cube at their position."""
    # Create model matrix at player's position
    model = Matrix44.from_translation(Vector3(player_state.pos))
    
    # Scale down the cube to represent a player (0.5 units tall)
    model = model * Matrix44.from_scale(Vector3([0.25, 0.5, 0.25]))
    
    mvp = projection * view * model
    prog['mvp'].write(mvp.astype('f4').tobytes())
    
    vao.render()

# In the render loop:
# for player_id, player_state in other_players.items():
#     render_player(player_state, prog, player_vao, projection, view)

## 5. Full Game Loop

Putting it all together:

In [ ]:
import pygame
import moderngl
from pyrr import Matrix44

# ... (Setup code: Pygame, ModernGL, Camera, Cube VAO, Shaders) ...

clock = pygame.time.Clock()
running = True

while running:
    dt = clock.tick(60) / 1000.0
    
    # Event handling
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.MOUSEMOTION:
            camera.process_mouse(event.rel[0], -event.rel[1])
    
    # Movement
    keys = pygame.key.get_pressed()
    camera.process_keyboard(keys, dt)
    
    # Network sync
    send_state(camera)  # Send our position
    other_players = receive_states()  # Get other players
    
    # Rendering
    ctx.clear(0.1, 0.1, 0.1)
    
    view = camera.get_view_matrix()
    projection = Matrix44.perspective_projection(45.0, 800/600, 0.1, 100.0)
    
    # Render the world (floor, walls, etc.)
    # ...
    
    # Render other players
    for player_id, player_state in other_players.items():
        render_player(player_state, prog, player_vao, projection, view)
    
    pygame.display.flip()

client.close()
pygame.quit()

## 🛠️ Challenge: The 3D Multiplayer Arena

Build a complete multiplayer 3D game:

1.  **Server**: Run the server to manage connections.
2.  **World**: Add a floor (large flat cube) and some walls.
3.  **Players**: Render other players as colored cubes.
4.  **HUD**: Display player count and your ID on screen.

**Bonus Challenges:**
- Add player names that float above their cubes
- Implement a simple "tag" mechanic (collision detection in 3D)
- Add projectiles that sync over the network

If you can see other players moving in your 3D world in real-time, **you've built a multiplayer 3D game from scratch**. 🏆

## 💡 Tips for Production

1.  **Use Threading**: Put network recv/send in a separate thread to avoid blocking the render loop.
2.  **Interpolation**: Smooth out player movement by interpolating between received positions.
3.  **Prediction**: Predict your own movement locally before the server confirms it.
4.  **Culling**: Only send player data for players within visible range.
5.  **Compression**: Use JSON instead of pickle, or implement binary protocols for better performance.

You now have all the tools to build the next Minecraft, Counter-Strike, or Quake. 🎮